In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def gauss_newton(y, H):
    x = np.linalg.solve(H.T@H,H.T@y)
    return x

In [ ]:
def huber(r,l,debug=False):
    m = np.shape(r)[0]
    cost = 0
    for i in range(m):
        if np.abs(r[i]) <= l:
            cost += 0.5*r[i]**2
            if debug:
                print("res ",i, " quad")
        else:
            cost += l*(np.abs(r[i])-0.5*l)
            if debug:
                print("res ",i, " lin")
    return cost

In [ ]:
def lev_marq(H,y,x0,l,trust):
    hessian = H.T@H + trust*np.diag(H.T@H)
    x = np.linalg.solve(hessian,H.T@y)
    if (huber(y-H@x,l)<huber(y-H@x0,l)) :#we apply the update
        x0 = x
        trust /= 10
    else : #no update, lower trust
        trust *= 10
    return (x0,trust)

In [ ]:
m = 10
n = 2
X = np.array([[1.],[1.]])
R = 1e-3*np.eye(m)
xi = np.reshape(np.arange(0,1,1/m,dtype=float),(m,1))
H = np.concatenate((xi,np.ones((m,1))),axis=1)
y = H@X + np.linalg.cholesky(R)@np.random.randn(m,1)
y[2,0] += 10
x_hat_gn = gauss_newton(y,H)
x_hat_lm = 1e6*np.random.randn(2,1)
trust = 1e-1
l_huber = 0.1
while True:
    (x_hat_lm_new,trust_new) = lev_marq(H,y,x_hat_lm,l_huber,trust)
    if (np.linalg.norm(x_hat_lm_new-x_hat_lm)<=1e-8):
        print(x_hat_lm)
        print(x_hat_lm_new)
        break
    else:
        print(x_hat_lm)
        print(x_hat_lm_new)
        x_hat_lm = x_hat_lm_new
        trust = np.copy(trust_new)
        
huber(y-H@x_hat_lm,l_huber,debug=True)

In [ ]:
x_plot = np.arange(0, 1, 0.01)
y_plot_lm = x_hat_lm[0,0]*x_plot+x_hat_lm[1,0]
y_plot_gn = x_hat_gn[0,0]*x_plot+x_hat_gn[1,0]
fig, ax = plt.subplots()
ax.plot(x_plot, y_plot_lm,'r')
ax.plot(x_plot, y_plot_gn,'g')
plt.scatter(xi,y)
print(x_hat_lm)
print(x_hat_gn)